In [1]:
import pandas as pd
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from sklearn.linear_model import LogisticRegression

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [4]:
train_df.head()

,Id,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,TARGET
0,57094,1.0,3,0,2016-05-19 18:06:27,A FABULOUS 3BR IN MIDTOWN WEST! PERFECT APAR...,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"['Laundry In Unit', 'No Fee', 'Elevator']",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium
1,33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,Renovated Kitchen and Bathroom!,55 River Drive South,"['Dogs Allowed', 'Cats Allowed', 'No Fee']",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium
2,60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,RARE AND BEST DEAL ON THE MARKET!!!! PERFECT S...,W 77 Street,"['Elevator', 'Hardwood Floors']",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low
3,53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,Newly renovated flex 2 apartment offers the ne...,John Street,"['Swimming Pool', 'Doorman', 'Elevator', 'Fitn...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low
4,592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,LOW FEE apartments do not come around like thi...,West 16th Street,"['Laundry in Building', 'Laundry in Unit', 'Di...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low


In [9]:
train_df.dtypes

Id                   int64
bathrooms          float64
bedrooms             int64
building_id         object
created             object
description         object
display_address     object
features            object
latitude           float64
listing_id           int64
longitude          float64
manager_id          object
photos              object
price                int64
street_address      object
TARGET              object
dtype: object

**id** - вряд ли хороший признак <br/>
**bathrooms** - хороший признак, малое разнообразие значений <br/>
**bedrooms** - хороший признак<br/>
**created** - дата, возможно хороший признак <br/>
**discription** - может получится выделить ключевые слова <br/>
**display_address** - пока не знаю <br/>
**features** - вероятно, полезные признаки <br/>
**latitude** - хороший признак <br/>
**listing_id** - вряд ли хороший признак <br/>
**longitude** - хороший признак <br/>
**manager_id** - не самый плохой признак, может попробую<br/>
**photo** - пока использовать не хватит сил<br/>
**price** - хороший признак <br/>
**street_address** - пока не знаю 

Преобразуем фичи из строки в массив фич

In [5]:
def makefeatures(x):
    object = x.replace("[","").replace("]","").replace("'","").split(",")
    return object

In [6]:
train_df.features = train_df.features.apply(lambda x: makefeatures(x))
test_df.features = test_df.features.apply(lambda x: makefeatures(x))

Преобразуем Target в циферки

In [12]:
classes = {
    'low': 0,
    'medium': 1,
    'high' : 2
       }

In [13]:
train_df["TARGET"] = train_df["TARGET"].apply(lambda x: classes[x])

In [14]:
X = train_df.drop(["Id","building_id","created","description","display_address","features","manager_id","photos","street_address","TARGET"],axis = 1)
Y = train_df["TARGET"]

In [15]:
Y_test = test_df.drop(["Id","building_id","created","description","display_address","features","manager_id","photos","street_address"],axis = 1)

In [16]:
X.head()

,bathrooms,bedrooms,latitude,listing_id,longitude,price
0,1.0,3,40.7647,7039994,-73.9918,4495
1,1.0,1,40.7275,7166774,-74.0322,2570
2,1.0,0,40.7798,6962716,-73.9751,1795
3,1.0,2,40.7081,7002458,-74.0065,3400
4,1.0,3,40.7416,7170465,-74.0025,5695


In [17]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score

In [18]:
def accuracy(y_real, y_pred):
  return np.where(y_pred == y_real)[0].shape[0] / y_real.shape[0]

def precision(y_real, y_pred):
    return np.where((y_pred == 1) & (y_real == 1))[0].shape[0] / np.where(y_pred == 1)[0].shape[0]

def recall(y_real, y_pred):
    return np.where((y_pred == 1) & (y_real == 1))[0].shape[0] / np.where(y_real == 1)[0].shape[0]

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [21]:
from sklearn.neighbors import KNeighborsClassifier


for column in X.columns:
  errors = np.mean(cross_val_score(KNeighborsClassifier(n_neighbors=5), X[column].values.reshape(-1, 1), Y, cv=5, scoring='accuracy'))
  print(f"Для {column} метрика {errors}")

Для bathrooms метрика 0.694696927328372
Для bedrooms метрика 0.5560766918461815
Для latitude метрика 0.6600473385770234
Для listing_id метрика 0.6573265140646892
Для longitude метрика 0.6542000859216119
Для price метрика 0.6565157424856817


In [ ]:
m_f2_log = []
from sklearn.metrics import fbeta_score
from itertools import product

weights = range(1, 11)

min_err_log=0

for a in [p for p in product(weights, repeat=3)]:
        a=list(a)
        m_log_reg_w = LogisticRegression(class_weight={0:a[0], 1:a[1], 2:a[2]}, max_iter=1000)
        m_log_reg_w.fit(X_train,y_train)
        errors = np.mean(cross_val_score(m_log_reg_w, X.values.reshape(-1, 6), Y, cv=5, scoring='f1_macro'))
        if errors>min_err_log:
            m_f2_log.append(errors)
            print(f"Построение Log модели при weight = {a} завершено. Метрика F2: {errors}")
            min_err_log=errors

Построение Log модели при weight = [1, 1, 1] завершено. Метрика F2: 0.27327594016321227</br>
Построение Log модели при weight = [1, 1, 2] завершено. Метрика F2: 0.27379482095611085</br>
Построение Log модели при weight = [1, 1, 3] завершено. Метрика F2: 0.3251895786807228</br>
Построение Log модели при weight = [1, 1, 4] завершено. Метрика F2: 0.3631181880467963</br>
Построение Log модели при weight = [1, 2, 4] завершено. Метрика F2: 0.36462846702959295</br>
Построение Log модели при weight = [1, 2, 5] завершено. Метрика F2: 0.37797044086931886</br>
Построение Log модели при weight = [1, 3, 8] завершено. Метрика F2: 0.3852229371801021</br>
Построение Log модели при weight = [2, 5, 8] завершено. Метрика F2: 0.39872530122616684</br>

In [23]:
reg = LogisticRegression(class_weight={0:2, 1:5, 2:9}, max_iter=1000)
reg.fit(X.values.reshape(-1, 6), Y)
y_pred = reg.predict(Y_test)

C:\Users\golev\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [24]:
from sklearn.model_selection import cross_val_predict

In [28]:
y_pred = cross_val_predict(LogisticRegression(class_weight={0:2, 1:5, 2:9}, max_iter=1000), X, Y,cv=3)

In [29]:
from sklearn.metrics import mean_squared_error as rmse

KNN

In [30]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train["price"].values.reshape(-1, 1),y_train)
y_pred = knn.predict(X_test["price"].values.reshape(-1, 1))
print(f"Accuracy : {accuracy( y_test, y_pred):.4f}")
print(f"Precision : {precision( y_test, y_pred):.4f}")
print(f"Recall : {recall(y_test, y_pred):.4f}")

Accuracy : 0.6575
Precision : 0.2769
Recall : 0.1501


LOGRES

In [31]:
reg = LogisticRegression()
reg.fit(X_train.values.reshape(-1, 6),y_train)
y_pred = reg.predict(X_test.values.reshape(-1, 6))
print(f"Accuracy : {accuracy( y_test, y_pred):.4f}")
print(f"Recall : {recall(y_test, y_pred):.4f}")

Accuracy : 0.7013
Recall : 0.0000


CatBoostRegressor

In [36]:
from catboost import CatBoostClassifier,CatBoostRegressor

In [49]:
cat = CatBoostClassifier(silent = True)
cat.fit(X_train.values.reshape(-1, 6),y_train)
y_pred = cat.predict(X_test.values.reshape(-1, 6))
print(f"Accuracy : {accuracy( y_test, y_pred.reshape(-1)):.4f}")
print(f"Precision : {precision( y_test, y_pred.reshape(-1)):.4f}")
print(f"Recall : {recall(y_test, y_pred.reshape(-1)):.4f}")

Accuracy : 0.7234
Precision : 0.4649
Recall : 0.2040


In [40]:
accuracies=0
best_rate = 0
best_depth = 0

for rate in np.linspace(0.1, 1,10):
  for depth in range(1,7,1):
    cbr = CatBoostRegressor(learning_rate = rate, depth = depth, iterations = 1000, silent = True, random_state = 42)
    cbr.fit(X_train.values.reshape(-1, 6),y_train)
    invest_y_pred = cbr.predict(X_test.values.reshape(-1, 6))
    if accuracy(y_test, y_pred.reshape(-1)) > accuracies : 
      accuracies = accuracy(y_test, y_pred.reshape(-1))
      best_rate = rate
      best_depth = depth
    print(".",end="")
  print("\n")
print(f"Best RMSE {accuracies:.4f} for rate {best_rate} for depth {best_depth}")

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

......

......

......

......

......

......

......

......

......

......

Best RMSE 0.7234 for rate 0.1 for depth 1


RandomForest

In [57]:
from sklearn.ensemble import RandomForestClassifier

In [59]:
rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train.values.reshape(-1, 6),y_train)
y_pred = rfc.predict(X_test.values.reshape(-1, 6))
print(f"Accuracy : {accuracy( y_test, y_pred.reshape(-1)):.4f}")
print(f"Precision : {precision( y_test, y_pred.reshape(-1)):.4f}")
print(f"Recall : {recall(y_test, y_pred.reshape(-1)):.4f}")

Accuracy : 0.7072
Precision : 0.4154
Recall : 0.2553


In [62]:
parametrs = { 'n_estimators': range (10, 51, 5),
              'max_depth': range (1,13, 2)}

In [60]:
from sklearn.model_selection import GridSearchCV

In [63]:
rfc = RandomForestClassifier(random_state=42)
grid = GridSearchCV(rfc, parametrs)
grid.fit(X_train.values.reshape(-1, 6),y_train)
y2_pred = grid.predict(X_test.values.reshape(-1, 6))
print(f"Accuracy : {accuracy( y_test, y_pred.reshape(-1)):.4f}")
print(f"Precision : {precision( y_test, y_pred.reshape(-1)):.4f}")
print(f"Recall : {recall(y_test, y_pred.reshape(-1)):.4f}")

Accuracy : 0.7072
Precision : 0.4154
Recall : 0.2553


Decision


In [64]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
y_pred = knn.predict(test_df.drop(["Id","building_id","created","description","display_address","features","manager_id","photos","street_address"],axis = 1))

In [120]:
Counter(y_pred.reshape(-1))

Counter({0: 9740, 1: 4396, 2: 670})

In [121]:
inv_classes = {
    0: 'low' ,
    1:'medium',
    2: 'high'
       }

In [122]:
y_pred = [inv_classes[x] for x in y_pred]
result  = pd.DataFrame( columns = ['Id', 'TARGET'])
result.Id = test_df.Id
result.TARGET = y_pred
result.to_csv("results.csv",index=False)

In [44]:
y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]], dtype=int64)